In [1]:
import requests
import pandas as pd
import re

In [2]:
def get_name(text):
    text = text.split(") ")[1]
    return text.split(" |")[0]

In [3]:
def get_position(text):
    pattern = r'\((.*?)\)'
    match = re.search(pattern, text)
    return match.group(1)

In [4]:
def get_flotilha(soup_object):
    """
    This functions gets a input like <span style="display:none;">punt_Yellow _0000003</span>
    and returns the series of this data.
    """
    if "Yellow" in str(soup_object) or "Blue" in str(soup_object) or "Red"in str(soup_object) or "Pink" in str(soup_object) or "White" in str(soup_object) or "Green" in str(soup_object) or "Emerald" in str(soup_object) \
    or "punt_ " in str(soup_object) or "Gold" in str(soup_object) or "Silver" in str(soup_object) or "Bronze" in str(soup_object):
        return "G"
    #DESCOMENTING BELOW IF THE CATEGORY IS FORMULA KITE
    # elif "QF" in str(soup_object):
    #     return "B"
    # elif "SF" in str(soup_object):
    #     return "P"
    # elif "Final" in str(soup_object):
    #     return "O"
    elif "Medal" in str(soup_object):
        return "MR"
    else:
        return "aa"

In [5]:
competitions = pd.read_excel("./Clusters-Sumulas.xlsx", sheet_name="Cluster 3")
competitions.head()

,Classe,Campeonato,Ano,Link,Descrição,Competição,the hague
0,Ilca 7,World Championship,2023,https://world-sailing-api.soticcloud.net/stats...,html,World Championship 2023,NaN
1,Ilca 6,World Championship,2023,https://world-sailing-api.soticcloud.net/stats...,html,World Championship 2023,NaN
2,IQFOIL Fem.,World Championship,2023,https://world-sailing-api.soticcloud.net/stats...,html,World Championship 2023,NaN
3,IQFOIL Masc.,World Championship,2023,https://world-sailing-api.soticcloud.net/stats...,html,World Championship 2023,NaN
4,49erFX,World Championship,2023,https://world-sailing-api.soticcloud.net/stats...,html,World Championship 2023,NaN


In [6]:
for i, competition in competitions.iterrows():
    
    COMPETICAO = competition["Competição"]
    CATEGORIA = competition["Classe"]

    r = requests.get(str(competition["Link"]))
    data = r.json()
    df_data = {
        "Id Resultado": [],
        "Id Competidor": [],
        "Nome Competidor": [],
        "Id Competição": [],
        "Classe Vela": [],
        "Pontuação Regata": [],
        "Flotilha": [],
        "Posição Geral": [],
        "Punição": [],
        "Pontuação Total": [],
        "Nett": [],
        "Nome Competição": [],
        "Regata": []    
    }

    for race in data["data"]["races"]:
        for result in race["raceResults"]:
            pontuacao = result["averagePoints"]
            nome = get_name(result["crew"]["_admin_name"])
            posicao = get_position(result["crew"]["_admin_name"])
            flotilha = get_flotilha(race["name"])
            totalpoints = result["crew"]["totalPoints"]
            net = result["crew"]["points"]
            flotilha = get_flotilha(race)
            regata = race["name"]
            df_data["Id Resultado"].append(None)
            df_data["Id Competidor"].append(None),
            df_data["Nome Competidor"].append(nome),
            df_data["Id Competição"].append(None),
            df_data["Classe Vela"].append(CATEGORIA),
            df_data["Pontuação Regata"].append(pontuacao),
            df_data["Posição Geral"].append(posicao),
            df_data["Flotilha"].append(flotilha)
            df_data["Punição"].append(None),
            df_data["Pontuação Total"].append(totalpoints),
            df_data["Nett"].append(net),
            df_data["Nome Competição"].append(COMPETICAO)
            df_data["Regata"].append(regata) 

    df = pd.DataFrame(df_data)
    # df.head()
    df.to_csv(f"../scraped-data-2024/c3/{COMPETICAO}{CATEGORIA}.csv", index=False)

## Agreggating results

In [7]:
import os

In [9]:
files = os.listdir("../scraped-data-2024/c3")

In [10]:
df_init = pd.read_csv("../scraped-data-2024/c3/"+files[0])
for file in files[1:]:
    df_init = pd.concat([df_init, pd.read_csv("../scraped-data-2024/c3/"+file)])
df_init.to_csv("../scraped-data-2024/cluster3.csv", index = False)    